<a href="https://colab.research.google.com/github/slunara/corporateProject/blob/main/simulationFunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions to compute

In [1]:
import numpy as np
import pandas as pd


def compute_customer_forecast(n_months, traffic, prob_prospect_generation,
                              prob_prospect_conversion,
                              prob_direct_customer_conversion, retention_prob, prob_existing_clients_conversion,
                              existing_customers):
    prospects = np.zeros(n_months)
    new_customers_from_prospects = np.zeros(n_months)
    new_customers_direct = np.zeros(n_months)
    total_existing_customers = np.zeros(n_months)

    prospects = traffic * prob_prospect_generation
    rows, cols = np.meshgrid(np.arange(n_months), np.arange(n_months), indexing="ij")
    M_conversion_matrix_ops = np.where(cols <= rows, prob_prospect_conversion[rows-cols], 0)
    new_customers_from_prospects = M_conversion_matrix_ops @ prospects
    new_customers_direct = traffic * prob_direct_customer_conversion
    M_retention_matrix_ops = np.where(cols <= rows, retention_prob[rows-cols], 0)
    retained_customers_from_prospects = M_retention_matrix_ops @ new_customers_from_prospects
    retained_customers_direct = M_retention_matrix_ops @ new_customers_direct
    existing_customers_vector = np.full(n_months, existing_customers * prob_existing_clients_conversion)
    total_existing_customers = existing_customers_vector + retained_customers_from_prospects + retained_customers_direct
    forecast_df = pd.DataFrame({
    "prospects": prospects,
    "new_customers_from_prospects": new_customers_from_prospects,  # Sum across rows
    "new_customers_direct": new_customers_direct,
    "total_existing_customers": total_existing_customers
    }, index=[f"Month {i+1}" for i in range(n_months)])

    return forecast_df

def compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df):

    total_revenue_df = pd.DataFrame(index=local_customer_forecast_df.index)

    total_revenue_df['revenue_local_new_from_prospects'] = local_customer_forecast_df['new_customers_from_prospects']*avg_ticket_df['local_avg_ticket_new_from_prospects'].iloc[0]
    total_revenue_df['revenue_local_new_from_direct'] = local_customer_forecast_df['new_customers_direct']*avg_ticket_df['local_avg_ticket_new_direct'].iloc[0]
    total_revenue_df['revenue_local_existing'] = local_customer_forecast_df['total_existing_customers']*avg_ticket_df['local_avg_ticket_existing'].iloc[0]
    total_revenue_df['revenue_local_total'] = total_revenue_df['revenue_local_new_from_prospects']+total_revenue_df['revenue_local_new_from_direct']+total_revenue_df['revenue_local_existing']

    total_revenue_df['revenue_tourist_new_from_prospects'] = tourist_customer_forecast_df['new_customers_from_prospects']*avg_ticket_df['tourist_avg_ticket_new_from_prospects'].iloc[0]
    total_revenue_df['revenue_tourist_new_from_direct'] = tourist_customer_forecast_df['new_customers_direct']*avg_ticket_df['tourist_avg_ticket_new_direct'].iloc[0]
    total_revenue_df['revenue_tourist_existing'] = tourist_customer_forecast_df['total_existing_customers']*avg_ticket_df['tourist_avg_ticket_existing'].iloc[0]
    total_revenue_df['revenue_tourist_total'] = total_revenue_df['revenue_tourist_new_from_prospects']+total_revenue_df['revenue_tourist_new_from_direct']+total_revenue_df['revenue_tourist_existing']

    total_revenue_df['revenue_total'] = total_revenue_df['revenue_local_total']+total_revenue_df['revenue_tourist_total']

    return total_revenue_df

def compute_kpi(traffic, existing_local_customers, existing_tourist_customers, total_revenue_df, local_customer_forecast_df, tourist_customer_forecast_df):

    # Compute local new closing ratio (new direct customers / total traffic)
    local_new_closing_ratio = local_customer_forecast_df['new_customers_direct'].sum() / traffic.sum()

    # Compute prospect closing ratio (new customers from prospects / total traffic)
    prospect_closing_ratio = local_customer_forecast_df['new_customers_from_prospects'].sum() / traffic.sum()

    # Compute local comeback rate (existing revenue / (existing customers + new direct customers))
    local_come_back = local_customer_forecast_df['total_existing_customers'].sum() / (
        existing_local_customers + local_customer_forecast_df['new_customers_direct'].sum()+ local_customer_forecast_df['new_customers_from_prospects'].sum()
    )

    # Compute tourist new closing ratio (new direct tourist customers / total traffic)
    tourist_new_closing_ratio = tourist_customer_forecast_df['new_customers_direct'].sum() / traffic.sum()

    # Compute tourist comeback rate (existing tourist customers / (previous existing tourists + new direct tourists))
    tourist_come_back = tourist_customer_forecast_df['total_existing_customers'].sum() / (
        existing_tourist_customers + tourist_customer_forecast_df['new_customers_direct'].sum()
    )

    # Create a DataFrame to store KPIs
    kpi_df = pd.DataFrame({
        "KPI": ["Local New Closing Ratio", "Prospect Closing Ratio", "Local Comeback Rate", "Tourist New Closing Ratio", "Tourist Comeback Rate"],
        "Value": [local_new_closing_ratio, prospect_closing_ratio, local_come_back, tourist_new_closing_ratio, tourist_come_back]
    })

    return kpi_df

def get_variable_to_change(sales_budget,sensitivity_df,baseline_total_revenue):

    # Calcular el porcentaje de crecimiento necesario
    growth_required = ((sales_budget - baseline_total_revenue) / baseline_total_revenue) * 100

    print(f"Alert: It is required a growth in sales {growth_required:.2f}% to reach sales budget.")
    incrementPerAction= (0.01*growth_required/sensitivity_df)*100
    incrementPerAction = incrementPerAction.rename(columns={"% Impact on Total Revenue": "Required Percentage Change"})

    return incrementPerAction

def compute_new_forecast(variable_To_Change, PO_change):
    """
    Computes the total revenue after modifying ONLY the given variable by PO_change.

    Parameters:
    - variable_To_Change (str): The name of the variable to be modified.
    - PO_change (float): The percentage change to be applied.

    Returns:
    - new_total_revenue (DataFrame): Updated revenue forecast.
    """
    # Create a copy of the original variable
    updated_variable_value = globals()[variable_To_Change] * (1 + PO_change/100)  # Apply % change

    # Debugging print statement
    print(f"🔍 Changing Variable: {variable_To_Change} | Original: {globals()[variable_To_Change]} -> New: {updated_variable_value}")

    # Recalculate forecasts using updated variable
    local_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        prob_prospect_generation if variable_To_Change != "prob_prospect_generation" else updated_variable_value,
        prob_prospect_conversion if variable_To_Change != "prob_prospect_conversion" else updated_variable_value,
        local_prob_direct_customer_conversion if variable_To_Change != "local_prob_direct_customer_conversion" else updated_variable_value,
        local_retention_prob if variable_To_Change != "local_retention_prob" else updated_variable_value,
        local_prob_existing_clients_conversion if variable_To_Change != "local_prob_existing_clients_conversion" else updated_variable_value,
        existing_local_customers if variable_To_Change != "existing_local_customers" else updated_variable_value
    )

    tourist_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        tourist_prob_prospect_generation if variable_To_Change != "tourist_prob_prospect_generation" else updated_variable_value,
        prob_prospect_conversion if variable_To_Change != "prob_prospect_conversion" else updated_variable_value,
        tourist_prob_direct_customer_conversion if variable_To_Change != "tourist_prob_direct_customer_conversion" else updated_variable_value,
        tourist_retention_prob if variable_To_Change != "tourist_retention_prob" else updated_variable_value,
        tourist_prob_existing_clients_conversion if variable_To_Change != "tourist_prob_existing_clients_conversion" else updated_variable_value,
        existing_tourist_customers if variable_To_Change != "existing_tourist_customers" else updated_variable_value
    )

    # Compute new total revenue
    new_total_revenue_df = compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df)

    return new_total_revenue_df,local_customer_forecast_df,tourist_customer_forecast_df


# Input

In [2]:
# ------------------------------ #
# Example usage for Local & Tourist Customers
# ------------------------------ #

n_months = 12
traffic = np.array([1000, 1200, 1100, 1300, 1250, 1400,1300, 1250, 1400,1300, 1250, 1400])  # traffic
prob_prospect_generation = 0.30
prob_prospect_conversion = np.array([0.3, 0.2, 0.1, 0, 0, 0,0,0,0,0,0,0])  # 30% of traffic converts into prospects

tourist_prob_prospect_generation = 0

local_prob_direct_customer_conversion = 0.1  # 10% of traffic converts directly into customers
tourist_prob_direct_customer_conversion = 0.05

local_retention_prob = np.array([0.2, 0.1, 0.06, 0.05, 0.04, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03])  # Retention probability for new customers
tourist_retention_prob = np.array([0.12, 0.6, 0.5, 0.3, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])  # Retention probability for new customers

local_prob_existing_clients_conversion = 0.03
tourist_prob_existing_clients_conversion = 0.01

existing_local_customers = 5000  # Existing local customers before the forecast
existing_tourist_customers = 1000  # Existing tourist customers before the forecast

local_avg_ticket_new_from_prospects = 50
local_avg_ticket_new_direct = 60
local_avg_ticket_existing = 40

tourist_avg_ticket_new_from_prospects = 0
tourist_avg_ticket_new_direct = 80
tourist_avg_ticket_existing = 45

sales_budget=500000


# Forecast all the stores

In [5]:
print(f"🔍 Sales Budget: {sales_budget}")

variables = {
    "prob_prospect_generation": prob_prospect_generation,
    "local_prob_prospect_conversion": prob_prospect_conversion,
    "local_prob_direct_customer_conversion": local_prob_direct_customer_conversion,
    "local_prob_existing_clients_conversion": local_prob_existing_clients_conversion,
    "local_retention_prob":local_retention_prob,

    "tourist_prob_direct_customer_conversion":tourist_prob_direct_customer_conversion,
    "tourist_prob_existing_clients_conversion": tourist_prob_existing_clients_conversion,
    "tourist_retention_prob":tourist_retention_prob
}


avg_ticket_df = pd.DataFrame({
    "local_avg_ticket_new_from_prospects": [local_avg_ticket_new_from_prospects],
    "local_avg_ticket_new_direct": [local_avg_ticket_new_direct],
    "local_avg_ticket_existing": [local_avg_ticket_existing],
    "tourist_avg_ticket_new_from_prospects": [tourist_avg_ticket_new_from_prospects],
    "tourist_avg_ticket_new_direct": [tourist_avg_ticket_new_direct],
    "tourist_avg_ticket_existing": [tourist_avg_ticket_existing]
})


# Compute forecasts for Local Customers
local_customer_forecast_df = compute_customer_forecast(
    n_months, traffic, prob_prospect_generation,prob_prospect_conversion,
    local_prob_direct_customer_conversion, local_retention_prob, local_prob_existing_clients_conversion,
    existing_local_customers
)

# Compute forecasts for Tourist Customers
tourist_customer_forecast_df = compute_customer_forecast(
    n_months, traffic, tourist_prob_prospect_generation,  prob_prospect_conversion,
    tourist_prob_direct_customer_conversion, tourist_retention_prob, tourist_prob_existing_clients_conversion,
    existing_tourist_customers
)

total_revenue=compute_revenue_forecast(local_customer_forecast_df,tourist_customer_forecast_df,avg_ticket_df)
sales_forecast=total_revenue['revenue_total'].sum()
print(f"🔍 Sales Forecast: {sales_forecast}")

initial_kpi_df=compute_kpi(traffic, existing_local_customers, existing_tourist_customers, total_revenue, local_customer_forecast_df, tourist_customer_forecast_df)
print(initial_kpi_df)

# Define baseline total revenue
baseline_total_revenue = total_revenue['revenue_total'].sum()
# Store sensitivity results
sensitivity_results = {}

# Loop through each variable to test its sensitivity
for var in variables.keys():
    # Create a copy of the original variables (fresh start each time)

    modified_variables = {k: v.copy() if isinstance(v, np.ndarray) else v for k, v in variables.items()}


    # Apply a 1% increase ONLY to the current variable
    modified_variables[var] = variables[var] * 1.01  # Increase by 1%


    # Compute forecasts using modified variables
    local_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        modified_variables["prob_prospect_generation"],
        modified_variables["local_prob_prospect_conversion"],
        modified_variables["local_prob_direct_customer_conversion"],
        modified_variables["local_retention_prob"],
        modified_variables["local_prob_existing_clients_conversion"],
        existing_local_customers
    )

    tourist_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        tourist_prob_prospect_generation,
        prob_prospect_conversion,
        modified_variables["tourist_prob_direct_customer_conversion"],
        modified_variables["tourist_retention_prob"],
        modified_variables["tourist_prob_existing_clients_conversion"],
        existing_tourist_customers
    )

    # Compute new total revenue
    new_total_revenue_df = compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df)

    # Extract sum of revenue
    new_total_revenue = new_total_revenue_df['revenue_total'].sum()


    # Calculate percentage impact
    sensitivity_results[var] = ((new_total_revenue - baseline_total_revenue) / baseline_total_revenue) * 100

# Convert results to DataFrame and display
sensitivity_df = pd.DataFrame.from_dict(sensitivity_results, orient='index', columns=["% Impact on Total Revenue"])
sensitivity_df = sensitivity_df.sort_values(by="% Impact on Total Revenue", ascending=False)

print(f"*Sensitivity Analysis*: {sensitivity_df}")

incrementPerAction=get_variable_to_change(sales_budget,sensitivity_df,baseline_total_revenue)

variable_To_Change = incrementPerAction.index[0]  # Get the first row's index (variable name)
PO_change = incrementPerAction.iloc[0, 0]  # Get the first row's value

new_total_revenue_df,local_customer_forecast_df,tourist_customer_forecast_df=compute_new_forecast(variable_To_Change, PO_change)
new_total_revenue = new_total_revenue_df['revenue_total'].sum()
print(f"New Sales Forecast: {new_total_revenue}")

final_kpi = compute_kpi(traffic, existing_local_customers, existing_tourist_customers, new_total_revenue_df, local_customer_forecast_df, tourist_customer_forecast_df)

print(final_kpi)

🔍 Sales Budget: 500000
🔍 Sales Forecast: 477881.37500000006
                         KPI     Value
0    Local New Closing Ratio  0.100000
1     Prospect Closing Ratio  0.169208
2        Local Comeback Rate  0.405775
3  Tourist New Closing Ratio  0.050000
4      Tourist Comeback Rate  0.643001
*Sensitivity Analysis*:                                           % Impact on Total Revenue
prob_prospect_generation                                   0.366555
local_prob_prospect_conversion                             0.366555
local_prob_direct_customer_conversion                      0.249556
tourist_prob_direct_customer_conversion                    0.221924
local_retention_prob                                       0.157681
local_prob_existing_clients_conversion                     0.150665
tourist_retention_prob                                     0.095114
tourist_prob_existing_clients_conversion                   0.011300
Alert: It is required a growth in sales 4.63% to reach sales budget.
🔍